In [10]:
#  Install the Python Requests library:
# `pip install requests`
import requests
import urllib.parse
import json
import pandas as pd


# URL = "https://www.google.com/travel/explore?tfs=CBwQAxoPag0IAhIJL20vMDE5NzhkGg9yDQgCEgkvbS8wMTk3OGRAAUgBcAKCAQ0I____________ARABmAEB&tfu=GgA&hl=es&gl=es&cur=eur"

URL = "https://www.google.com/travel/explore?q=flights%20from%20MAD&hl=es&gl=es&cur=eur"

encoded_url = urllib.parse.quote(URL)

content = ""

def send_request():
    response = requests.get(
        url='https://app.scrapingbee.com/api/v1/',
        params={
            'api_key': 'VCKXJYS1WXGF28LE1KJWD6KP0SY1ULG5D6TUADEEZCNNHEK9O24G5J9LL8SRICLRPBKLJIFCYGUO5BW6',
            'url': URL, 
            'wait': '3000',
            'cookies': 'CONSENT=YES+;',
            'wait_for': '.SD4Ugf',
            'extract_rules': '{"flights" : {        "selector": "ol > li",        "type": "list",        "output": {            "destination": "h3",            "date": ".xyc80b",             "price": ".MJg7fb > span",            "layovers": ".nx0jzf",            "duration": ".Xq1DAb",        }    },}',
            'custom_google': 'true',
            'block_resources': 'false',
            'premium_proxy': 'true',
        },
        
    )
    content = response.content
    print('Response HTTP Status Code: ', response.status_code)
    print('Response HTTP Response Body: ', content)

    return content


Response HTTP Status Code:  200
Response HTTP Response Body:  b'{"flights": [{"destination": "Par\xc3\xads", "date": "", "price": "46 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 10 min"}, {"destination": "Londres", "date": "", "price": "59 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 40 min"}, {"destination": "Roma", "date": "", "price": "58 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 25 min"}, {"destination": "Mil\xc3\xa1n", "date": "", "price": "40 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 14 min"}, {"destination": "Lisboa", "date": "", "price": "39 \xe2\x82\xac", "layovers": "Directo", "duration": "1 h 25 min"}, {"destination": "Dubl\xc3\xadn", "date": "", "price": "57 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 40 min"}, {"destination": "Berl\xc3\xadn", "date": "", "price": "96 \xe2\x82\xac", "layovers": "Directo", "duration": "3 h"}, {"destination": "Venecia", "date": "", "price": "62 \xe2\x82\xac", "layovers": "Directo", "du

In [11]:
import pandas as pd

# Your JSON response
json_response = send_request()

# Decode the bytes and load JSON
data = json.loads(json_response.decode('utf-8'))

df = pd.DataFrame(data['flights'])

# Remove rows with empty prices
df = df[df['price'] != '']

# Convert the 'price' column to numeric (removing the euro symbol)
df['price'] = df['price'].str.replace(' €', '').astype(int)

# Sort the DataFrame by the 'price' column
df_sorted = df.sort_values(by='price')

# Reset the index after sorting
df_sorted.reset_index(drop=True, inplace=True)

# Add the euro sign to the column name 'price'
df_sorted.rename(columns={'price': 'price (€)'}, inplace=True)

# Print the DataFrame
print(df_sorted)

Response HTTP Status Code:  200
Response HTTP Response Body:  b'{"flights": [{"destination": "Par\xc3\xads", "date": "", "price": "46 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 10 min"}, {"destination": "Londres", "date": "", "price": "59 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 40 min"}, {"destination": "Roma", "date": "", "price": "58 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 25 min"}, {"destination": "Mil\xc3\xa1n", "date": "", "price": "40 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 14 min"}, {"destination": "Lisboa", "date": "", "price": "39 \xe2\x82\xac", "layovers": "Directo", "duration": "1 h 25 min"}, {"destination": "Dubl\xc3\xadn", "date": "", "price": "57 \xe2\x82\xac", "layovers": "Directo", "duration": "2 h 40 min"}, {"destination": "Berl\xc3\xadn", "date": "", "price": "96 \xe2\x82\xac", "layovers": "Directo", "duration": "3 h"}, {"destination": "Venecia", "date": "", "price": "62 \xe2\x82\xac", "layovers": "Directo", "du

In [4]:
from geopy.geocoders import Nominatim

# Initialize geolocator
geolocator = Nominatim(user_agent="flight_prices")

# Function to get latitude and longitude
def get_coordinates(location):
    try:
        geo_location = geolocator.geocode(location)
        if geo_location:
            return geo_location.latitude, geo_location.longitude
        else:
            return None, None
    except:
        return None, None

# Apply the function to each destination
df['latitude'], df['longitude'] = zip(*df['destination'].apply(get_coordinates))

# Print the DataFrame with latitude and longitude
print(df)


           destination       date  price  layovers     duration   latitude  \
0            Barcelona    4–7 abr     44   Directo   1 h 35 min  41.382894   
1                París  14–17 jun     65   Directo   2 h 40 min  48.853495   
2            Ámsterdam    4–7 abr    114   Directo   3 h 10 min  52.373080   
3                 Roma  12–15 abr     73   Directo   2 h 30 min  41.893320   
4                Milán  25–29 abr     54   Directo   2 h 35 min  45.464194   
5               Lisboa  27–30 jun     59   Directo   1 h 20 min  38.707751   
6               Dublín  25–29 abr     77   Directo    3 h 5 min  53.349379   
7               Madrid   7–10 mar     49   Directo   1 h 15 min  40.416705   
8                Viena    4–7 abr     74   Directo   3 h 20 min  48.208354   
9               Berlín   7–10 jun    123   Directo   3 h 30 min  52.517037   
10               Praga  23–26 may    105   Directo   3 h 20 min  50.087465   
11            Budapest   9–11 mar     99   Directo   3 h 35 min 

In [5]:
import folium

# Create a base map centered around a specific location (e.g., world center)
map_center = [0, 0]
world_map = folium.Map(location=map_center, zoom_start=2)

# Add markers for each destination with its price
for index, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],  # Use latitude and longitude from DataFrame
        popup=f"{row['destination']} - {row['price']} €",
        icon=folium.Icon(color='blue')
    ).add_to(world_map)

# Display the map
world_map
